In [10]:
import carla
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.distributions import Normal
import csv

In [2]:
class PPO_MAML_Network(nn.Module):
    def __init__(self, state_dim, action_dim):
        super(PPO_MAML_Network, self).__init__()
        self.actor = nn.Sequential(
            nn.Linear(state_dim, 64),
            nn.ReLU(),
            nn.Linear(64, 64),
            nn.ReLU(),
            nn.Linear(64, action_dim)
        )
        self.critic = nn.Sequential(
            nn.Linear(state_dim, 64),
            nn.ReLU(),
            nn.Linear(64, 64),
            nn.ReLU(),
            nn.Linear(64, 1)
        )

    def forward(self, state):
        action_mean = self.actor(state)
        value = self.critic(state)
        return action_mean, value

In [3]:
class CarlaEnvironment:
    def __init__(self, world, vehicle_blueprint, waypoints):
        self.world = world
        self.vehicle_blueprint = vehicle_blueprint
        self.waypoints = waypoints
        self.reset()
        self.actorlist = []
    def reset(self):
        self.vehicle = self.world.spawn_actor(self.vehicle_blueprint, carla.Transform(carla.Location(x=-23.6,y=137.5,z=1),carla.Rotation(yaw=0)))
        self.current_waypoint = 0
        return self.get_state()

    def step(self, action):
        # Apply actions: Throttle, Steering, Brake
        self.vehicle.apply_control(carla.VehicleControl(throttle=action[0], steer=action[1], brake=action[2]))
        self.actorlist.append(self.vehicle)

        # Get next state
        next_state = self.get_state()

        # Calculate reward
        reward = self.calculate_reward(next_state)

        # Check if the episode is done
        done = self.is_done(next_state)

        return next_state, reward, done

#     def get_state(self):
#         curr_vel = self.vehicle.get_velocity()
#         curr_yaw = self.vehicle.get_transform().rotation.yaw
#         waypoint = self.waypoints[self.current_waypoint]
#         return np.array([curr_vel, curr_yaw - waypoint[1], curr_yaw - waypoint[2], waypoint[3]])
    
    def get_state(self):
        state = []
        curr_yaw = self.vehicle.get_transform().rotation.yaw
        

        for i in range(10):
            if self.current_waypoint + i < len(self.waypoints):
                waypoint = self.waypoints[self.current_waypoint + i]
                curr_vel = self.vehicle.get_velocity()
                curr_loc = self.vehicle.get_location()
                distance = curr_loc.distance(x=waypoint[4],y=waypoint[5])
                state.extend([curr_vel, curr_yaw - waypoint[1], curr_yaw - waypoint[2], distance ])
            else:
                state.extend([0, 0, 0, 0])

        return np.array(state)

    def generate_waypoints(self):
        # Implement waypoint generation logic
        pass

    def calculate_reward(self, state):
        reward = -(np.linalg.norm(state[0][0] - self.waypoints[self.current_waypoint][0]) - (np.linalg.norm (state[0][1])) - (np.linalg.norm ( state[0][2])) - (np.linalg.norm (state[0][3])) + self.current_waypoint)
        return reward

    def is_done(self, state):
        if np.abs(state[3]) < 0.20:  # If the vehicle is close to the next waypoint
            self.current_waypoint += 1

        if self.current_waypoint >= len(self.waypoints):
            return True

        return False

    def set_weather(self, weather):
        self.world.set_weather(weather)
        
    def destroy(self):
        for actor in self.actorlist:
            actor.destroy()
        print("All Cleared")

In [12]:
def sample_weather_conditions(num_weather_conditions):
    weather_types = [carla.WeatherParameters.ClearNoon, carla.WeatherParameters.WetNoon,
                     carla.WeatherParameters.WetCloudyNoon, carla.WeatherParameters.HeavyRainNoon]
    return np.random.choice(weather_types, num_weather_conditions)

def sample_trajectories(env, model, num_episodes=10, max_episode_length=1000):
    trajectories = []

    for _ in range(num_episodes):
        state = env.reset()
        trajectory = []

        for _ in range(max_episode_length):
            state_tensor = torch.tensor(state, dtype=torch.float32)
            action_mean, _ = model(state_tensor)
            action_distribution = Normal(action_mean, torch.tensor(0.1))
            action = action_distribution.sample().detach().numpy()
            next_state, reward, done = env.step(action)
            trajectory.append((state, action, reward, next_state, done))

            state = next_state

            if done:
                break

        trajectories.append(trajectory)

    return trajectories

def compute_advantages_and_returns(trajectories, model, gamma=0.99, lam=0.95):
    advantages = []
    returns = []

    for trajectory in trajectories:
        traj_advantages = []
        traj_returns = []
        prev_value = 0
        prev_advantage = 0

        for state, _, reward, next_state, done in reversed(trajectory):
            state_tensor = torch.tensor(state, dtype=torch.float32)
            next_state_tensor = torch.tensor(next_state, dtype=torch.float32)

            _, value = model(state_tensor)
            _, next_value = model(next_state_tensor)

            if done:
                delta = reward - prev_value
                traj_advantages.append(delta)
                traj_returns.append(reward)
            else:
                delta = reward + gamma * next_value - prev_value
                advantage = delta + gamma * lam * prev_advantage
                traj_advantages.append(advantage)
                traj_returns.append(reward + gamma * prev_advantage)

            prev_value = value
            prev_advantage = advantage

        traj_advantages.reverse()
        traj_returns.reverse()

        advantages.extend(traj_advantages)
        returns.extend(traj_returns)

    return advantages, returns

def compute_losses(trajectories, model, advantages, returns, clip_ratio=0.2, value_coeff=0.5, entropy_coeff=0.01):
    state_tensor = torch.tensor([state for trajectory in trajectories for state, _, _, _, _ in trajectory], dtype=torch.float32)
    action_tensor = torch.tensor([action for trajectory in trajectories for _, action, _, _, _ in trajectory], dtype=torch.float32)
    advantage_tensor = torch.tensor(advantages, dtype=torch.float32)
    return_tensor = torch.tensor(returns, dtype=torch.float32)

    action_mean, value = model(state_tensor)
    action_distribution = Normal(action_mean, torch.tensor(0.1))
    log_probs = action_distribution.log_prob(action_tensor)

    old_action_mean, old_value = model(state_tensor)
    old_action_distribution = Normal(old_action_mean, torch.tensor(0.1))
    old_log_probs = old_action_distribution.log_prob(action_tensor).detach()

    ratio = torch.exp(log_probs - old_log_probs)
    clipped_ratio = torch.clamp(ratio, 1 - clip_ratio, 1 + clip_ratio)
    policy_loss = -torch.min(ratio * advantage_tensor, clipped_ratio * advantage_tensor).mean()

    value_loss = value_coeff * (return_tensor - value).pow(2).mean()

    entropy_loss = entropy_coeff * action_distribution.entropy().mean()

    total_loss = policy_loss + value_loss - entropy_loss

    return total_loss

def train_ppo_maml(env, state_dim, action_dim, lr=1e-4, n_iters=1000000, inner_loop_steps=5, num_weather_conditions=4):
    model = PPO_MAML_Network(state_dim, action_dim)
    optimizer = optim.Adam(model.parameters(), lr=lr)

    for iteration in range(n_iters):
        weather_conditions = sample_weather_conditions(num_weather_conditions)
        task_gradients = []

        for weather in weather_conditions:
            env.set_weather(weather)
            trajectories = sample_trajectories(env, model)
            advantages, returns = compute_advantages_and_returns(trajectories, model)
            inner_loop_gradients = []

            for _ in range(inner_loop_steps):
                total_loss = compute_losses(trajectories, model, advantages, returns)
                optimizer.zero_grad()
                total_loss.backward()
                inner_loop_gradients.append([p.grad.clone() for p in model.parameters()])

            task_gradients.append(inner_loop_gradients)

        optimizer.zero_grad()
        for i, param in enumerate(model.parameters()):
            mean_gradient = torch.mean(torch.stack([g[i] for g in task_gradients]), dim=0)
            param.grad = mean_gradient
        optimizer.step()

        if iteration % 1000 == 0:
            print("Iteration: ", iteration)
            
def read_waypoints(file_path):
    waypoints = []
    with open(file_path, 'r') as csvfile:
        csvreader = csv.reader(csvfile)
        for row in csvreader:
            x, y, yaw, r = map(float, row)
            waypoints.append((x, y, yaw, r))
            
    return waypoints                        

In [13]:
if __name__ == '__main__':
    
    client = carla.Client('localhost', 2000)
    client.set_timeout(2.0)

    world = client.get_world()
    blueprint_library = world.get_blueprint_library()
    vehicle_blueprint = blueprint_library.filter('vehicle.jeep.wrangler_rubicon')[0]
    
    waypoints = read_waypoints("waypoint_distance_data.csv")
    env = CarlaEnvironment(world, vehicle_blueprint, waypoints)  # Implement CarlaEnvironment with the required state and action spaces

    state_dim = 40
    action_dim = 3

    train_ppo_maml(env, state_dim, action_dim)

ArgumentError: Python argument types in
    Location.distance(Location, float)
did not match C++ signature:
    distance(carla::geom::Location {lvalue}, carla::geom::Location location)